In [ ]:
from __future__ import division
import pandas as pd
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import MaxNLocator

from matplotlib.ticker import FormatStrFormatter

from scipy import stats
import scipy.stats as sstat
import scipy.signal as ssig
import h5py
from mpl_toolkits.mplot3d import Axes3D
import os
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA as sklearnPCA
import re

# import ephys_unit_analysis as ena
import mz_ephys_unit_analysis as mz_ena
import mz_unit_dur_freq as mz_udf

#import resampy
import fnmatch
import seaborn as sns
%matplotlib inline
%load_ext autoreload
%autoreload 2
mpl.rcParams['pdf.fonttype'] = 42 
mpl.rcParams['font.sans-serif']=['Arial', 'Helvetica','Bitstream Vera Sans', 'DejaVu Sans', 'Lucida Grande', 
                                 'Verdana', 'Geneva', 'Lucid', 'Avant Garde', 'sans-serif']  

sns.despine()

In [2]:
# for publication quality plots
def set_pub_plots(pal=sns.blend_palette(['cyan', 'magenta','gray','crimson','purple'], 5)):
    sns.set_style("white")
    sns.set_palette(pal)
    sns.set_context("poster", font_scale=1.5, rc={"lines.linewidth": 2.5, "axes.linewidth":1.5, 'figure.facecolor': 'white'}) 
    sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})
    plt.rcParams['axes.linewidth'] = 1.5

rc_pub={'font.size': 15, 'axes.labelsize': 15, 'legend.fontsize': 10.0, 
        'axes.titlesize': 25, 'xtick.labelsize': 15, 'ytick.labelsize': 15, 
        'axes.linewidth':1.0, 'lines.linewidth': 2.5,
        'xtick.color': 'black', 'ytick.color': 'black', 'axes.edgecolor': 'black',
        'axes.labelcolor':'black','text.color':'black'}
# to restore the defaults, call plt.rcdefaults() 

set_pub_plots()

In [ ]:
pal=sns.blend_palette(['cyan', 'magenta','gray','crimson','purple'], 5)
sns.palplot(pal)
sns.set_palette(pal)

---

# First, load in the previously saved dataFrame

In [ ]:
psth_df = pd.read_pickle(r"C:\Users\AChub_Lab\Desktop\DIA_CQuinn\V1HPC_psth.pkl")
psth_df.head()

In [ ]:
print(psth_df.et.unique())
print(psth_df.region.unique())
print(psth_df.stim.unique())

In [ ]:
psth_df.groupby('stim')['et'].nunique()

---

# Selecting only the visually responsive units
This requires a __`user input`__ to run

In [7]:
vr_resp_df = psth_df[psth_df.r_group_type == 'excited']

In [8]:
stim_choice = 'post'

if stim_choice != 'novel':
    vis_resp_df = vr_resp_df[vr_resp_df['stim'] == stim_choice]
else:
    bad_ets = []#'HP21','HP31']
    vis_resp_df = vr_resp_df[vr_resp_df['stim'] == stim_choice]
    v1_updated = vis_resp_df[(~vis_resp_df['et'].isin(bad_ets)) & (vis_resp_df['region']=='v1')] #excludes v1 units that are from ets
    hpc_updated = vis_resp_df[vis_resp_df['region']=='hippo']
    vis_resp_df = pd.concat([v1_updated, hpc_updated])

# Separating by group and moving to arrays

In [ ]:
print(f'all excited units shape for {stim_choice}')

group_A_df = vis_resp_df[vis_resp_df['region'] == 'v1']
group_B_df = vis_resp_df[vis_resp_df['region'] == 'hippo']

A_array = np.array(group_A_df.pivot(index='cuid', columns='times', values='zscore'))
print(f'v1: {A_array.shape}') #this is [units x times] and the values are above

B_array = np.array(group_B_df.pivot(index='cuid', columns='times', values='zscore'))
print(f'hpc: {B_array.shape}') #this is [units x times] and the values are above

---

# Plotting heatmaps of all units (vr)

In [258]:
#this checks each et for the units that are included
# sp_dims = (12, 4) #(width, height)
# v_max, v_min = 5, -1
# xvalues = np.arange(start=0.4, stop=1.7, step=0.2)
# y_max, y_min = 6, -2

# for t,tt in group_A_df.groupby('et'):
#     is_sorted = False
#     v1_all = tt.pivot(index='cuid', columns='times', values='zscore')
#     hip_all = tt.pivot(index='cuid', columns='times', values='zscore')
#     if is_sorted:
#         v1_km_np = v1_all.values[ np.argsort(np.mean(v1_all.values[:,50:80], axis = 1) )] #if I want sorted activity
#         hip_km_np = hip_all.values[ np.argsort(np.mean(hip_all.values[:,50:80], axis = 1) )] #if I want sorted activity
#     else:
#         v1_km_np = v1_all.values
#         hip_km_np = hip_all.values

#     f, ax1 = plt.subplots(1,1, sharex='col', figsize=(10,3))
#     # Heatmap for A
#     sns.heatmap(v1_km_np, cmap = 'afmhot', xticklabels=20, yticklabels=False, vmax=v_max, center = 2, vmin = v_min, robust = True, ax=ax1)
#     ax1.set_xlabel('Time (ms)')
#     ax1.set_ylabel(str(v1_km_np.shape[0]))
#     ax1.set_xlim(0.4*100,1.6*100)
#     ax1.set_title(t)
#     plt.show()

In [10]:
is_sorted = True

v1_all = group_A_df.pivot(index='cuid', columns='times', values='zscore')
hip_all = group_B_df.pivot(index='cuid', columns='times', values='zscore')

if is_sorted:
    v1_km_np = v1_all.values[ np.argsort(np.mean(v1_all.values[:,50:80], axis = 1) )] #if I want sorted activity
    hip_km_np = hip_all.values[ np.argsort(np.mean(hip_all.values[:,50:80], axis = 1) )] #if I want sorted activity
else:
    v1_km_np = v1_all.values
    hip_km_np = hip_all.values

In [11]:
sp_dims = (12, 4) #(width, height)
v_max, v_min = 5, -1
xvalues = np.arange(start=0.4, stop=1.7, step=0.2)
y_max, y_min = 6, -2

In [ ]:
f, (ax1, ax2) = plt.subplots(1,2, sharex='col', figsize=sp_dims)
f.tight_layout()
plt.suptitle(stim_choice)

# Heatmap for A
sns.heatmap(v1_km_np, 
            cmap = 'afmhot', xticklabels=20, yticklabels=False, 
            vmax=v_max, center = 2, vmin = v_min, robust = True, ax=ax1)
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel(str(v1_km_np.shape[0]))
ax1.set_xlim(0.4*100,1.6*100)
ax1.set_title('V1')
cbar = ax1.collections[0].colorbar
cbar.remove()

#Heatmap for B
sns.heatmap(hip_km_np, 
            cmap = 'afmhot', xticklabels=20, yticklabels=False, 
            vmax=v_max, center = 2, vmin = v_min, robust = True, ax=ax2)
ax2.set_xlabel('Time (ms)')
ax2.set_ylabel(str(hip_km_np.shape[0]))
ax2.set_xlim(0.4*100,1.6*100)
ax2.set_title('Hippo')
cbar = ax2.collections[0].colorbar
cbar.remove()

# Change the end of this next line with a new file name!!
# plt.savefig(r"C:\Users\AChub_Lab\Desktop\DIA_CQuinn\figure_panels\redo_v1hpc_mainfigure\novel_V1HPC_heatmap_excitedUnits.pdf", transparent=True)

plt.show()

In [ ]:
f, ax = plt.subplots(1,1, sharex='col', figsize=(10,3))

g2 = sns.lineplot(x = 'times', y = 'ztc', estimator = 'mean', data = pd.concat([group_A_df,group_B_df]), hue='region',
                  color = 'grey', ax=ax, errorbar=None)
ax.set(xlabel='Time (s)', ylabel='Zscore')
ax.set_title(stim_choice)
ax.axhline(y=0, color='k', linestyle="--", linewidth=1.0, alpha=0.75)
ax.set_xlim(0.4,1.6)
ax.set_xticks([0.4,0.6,0.8,1.0,1.2,1.4,1.6])
ax.set_ylim(-2,22)
ax.set_yticks([0,5,10,15,20])

sns.despine()
# Change the end of this next line with a new file name!!
# plt.savefig(r"C:\Users\AChub_Lab\Desktop\DIA_CQuinn\figure_panels\redo_v1hpc_mainfigure\novel_V1HPC_line_excitedUnits.pdf", transparent=True)

plt.show()

---

# Plotting all of the detected peaks (units with N or more peaks)

In [14]:

num_cycles_required = 3
is_sorted = False


In [15]:
durationA, indA, cuidA_ls = [], [], []
for unit in range(A_array.shape[0]):
    yy = A_array[unit]
    dur, peak_ind = mz_udf._duration_peaks_unit(yy)
    if len(peak_ind) >= num_cycles_required:
        durationA.append(dur)
        indA.append(peak_ind)
        cuidA_ls.append(yy)

durationB, indB, cuidB_ls = [], [], []
for unit in range(B_array.shape[0]):
    yy = B_array[unit]
    dur, peak_ind = mz_udf._duration_peaks_unit(yy)
    if len(peak_ind) >= num_cycles_required:
        durationB.append(dur)
        indB.append(peak_ind)
        cuidB_ls.append(yy)

indA_array = np.concatenate(indA, axis=0)
indB_array = np.concatenate(indB, axis=0)

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 10.0 # x dimension
fig_size[1] = 4.0 # y dimension
plt.axvspan(50,70, color='grey', alpha=0.2)

sns.histplot(indA_array, binwidth=3, linewidth=0, stat='count', color='cyan', kde=True, kde_kws={'bw_adjust' : 0.12})
sns.histplot(indB_array, binwidth=3, linewidth=0, stat='count', color='magenta', kde=True, kde_kws={'bw_adjust' : 0.12})
# sns.kdeplot(indA_array, bw_method=0.05, color='cyan', fill=True)
# sns.kdeplot(indB_array, bw_method=0.05, color='magenta', fill=True)

plt.title(str('kde peak times: '+stim_choice))
plt.xlim([50,110])
plt.xticks(ticks=[50,60,70,80,90,100], labels=[0.5,0.6,0.7,0.8,0.9,1.0])
# plt.yticks(ticks=[0,0.02,0.04])
plt.xlabel('Time (s)')

sns.despine()
# Change the end of this next line with a new file name!!
# plt.savefig(r"C:\Users\AChub_Lab\Desktop\DIA_CQuinn\figure_panels\redo_v1hpc_mainfigure\post_V1HPC_3peaks.pdf", transparent=True)

plt.show()

In [268]:
cuidA_arr = np.array(cuidA_ls)
cuidB_arr = np.array(cuidB_ls)

is_sorted = False #dont change this, the code will break

if is_sorted:
    v1_km_np = np.argsort(np.mean(cuidA_arr[:,50:80], axis = 1)) #if I want sorted activity
    hip_km_np = np.argsort(np.mean(cuidB_arr[:,50:80], axis = 1)) #if I want sorted activity
else:
    v1_km_np = cuidA_arr
    hip_km_np = cuidB_arr

In [ ]:
f, (ax1, ax2) = plt.subplots(1,2, sharex='col', figsize=sp_dims)
f.tight_layout()
plt.suptitle(f'{stim_choice} 3 cycles')

# Heatmap for A
sns.heatmap(v1_km_np, 
            cmap = 'afmhot', xticklabels=20, yticklabels=False, 
            vmax=v_max, center = 2, vmin = v_min, robust = True, ax=ax1)
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel(str(v1_km_np.shape[0]))
ax1.set_xlim(0.4*100,1.6*100)
ax1.set_title('V1')
cbar = ax1.collections[0].colorbar
cbar.remove()

#Heatmap for B
sns.heatmap(hip_km_np, 
            cmap = 'afmhot', xticklabels=20, yticklabels=False, 
            vmax=v_max, center = 2, vmin = v_min, robust = True, ax=ax2)
ax2.set_xlabel('Time (ms)')
ax2.set_ylabel(str(hip_km_np.shape[0]))
ax2.set_xlim(0.4*100,1.6*100)
ax2.set_title('Hippo')
cbar = ax2.collections[0].colorbar
cbar.remove()

# Change the end of this next line with a new file name!!
# plt.savefig(r"C:\Users\AChub_Lab\Desktop\DIA_CQuinn\figure_panels\redo_v1hpc_mainfigure\post_V1HPC_heatmap_3cycles.pdf", transparent=True)

plt.show()

---

# Oscillation duration

In [ ]:
durA_df = pd.DataFrame(tuple(zip(durationA,(['v1']*len(durationA)))),columns=['dur','group'])
durB_df = pd.DataFrame(tuple(zip(durationB,(['hip']*len(durationB)))),columns=['dur','group'])
all_dur_df=pd.concat([durA_df,durB_df])
print(durA_df.shape, durB_df.shape)
all_dur_df.head()

In [ ]:
#removes the NaN values from the above dataframe
all_dur_df=all_dur_df[~all_dur_df.dur.isna()]
all_dur_df.shape

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 7.0 # x dimension
fig_size[1] = 4.0 # y dimension

ax1= plt.subplot(111)
sns.kdeplot(data=all_dur_df, x="dur", 
            hue="group",
            cumulative=True, 
            common_norm=False,
            common_grid=True,
            bw_method=0.5,
            legend=True
           )

plt.title(stim_choice)
plt.xlabel('Time (s)')
plt.xticks([0,0.5,1.0,1.5])
plt.xlim([-0.1,1.6])
plt.yticks([0,0.5,1.0])

sns.despine()
# Change the end of this next line with a new file name!!
# plt.savefig(r"C:\Users\AChub_Lab\Desktop\DIA_CQuinn\figure_panels\redo_v1hpc_mainfigure\novel_V1HPC_durationKDE_excitedUnits.pdf", transparent=True)

plt.show()

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 2.0 # x dimension
fig_size[1] = 5.0 # y dimension

ax1=plt.subplot(111)
sns.barplot(data=all_dur_df, x='group', y='dur', order=["v1","hip"], errorbar=('ci', 68))

plt.ylabel('Time (s)')
plt.xlabel('')
ax1.set_xticklabels(['V1','HPC'])
plt.yticks(ticks=[0,0.1,0.2,0.3,0.4,0.5])
plt.ylim([0,0.55])
plt.title(stim_choice)

sns.despine()
# Change the end of this next line with a new file name!!
# plt.savefig(r"C:\Users\AChub_Lab\Desktop\DIA_CQuinn\figure_panels\redo_v1hpc_mainfigure\novel_V1HPC_durationMean_excitedUnits.pdf", transparent=True)

plt.show()

# Run the stats on the oscillation duration

In [ ]:
result = sstat.ks_2samp(all_dur_df[all_dur_df['group']=='v1'].dur, all_dur_df[all_dur_df['group']=='hip'].dur)
print('Oscillation Duration')
print(stim_choice)
print('v1 units: {0} --- hip units: {1}'.format(len(indA),len(indB)))
print(result)

In [ ]:
print('Oscillation Duration')
print(stim_choice)

print('Mean of WT: {0}'.format(np.mean(all_dur_df[all_dur_df['group']=='v1'].dur)))
print('Mean of FX: {0}'.format(np.mean(all_dur_df[all_dur_df['group']=='hip'].dur)))

result = sstat.ttest_ind(all_dur_df[all_dur_df['group']=='v1'].dur, all_dur_df[all_dur_df['group']=='hip'].dur)
print(result)

---

---

---